In [2]:
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input

from tensorflow.keras.applications import VGG19
from keras.layers import Dense, Dropout, Flatten, Activation
from keras import regularizers
from keras.models import Model

from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping, TensorBoard

In [ ]:
# Check GPU availability
print(tf.__version__)
print(tf.config.list_physical_devices())

In [3]:
data = '/kaggle/input/three-facial-emotions-dataset'

# Init the Image Data Generator
train_data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    preprocessing_function=preprocess_input
)

# Train Images
train_generator = train_data_gen.flow_from_directory(
        f'{data}/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training'
)

# Validation images (From Train Images)
validation_generator = train_data_gen.flow_from_directory(
        f'{data}/train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
)

Found 12000 images belonging to 3 classes.
Found 3000 images belonging to 3 classes.


In [ ]:
# Load the VGG19 model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False
    
# Unfreeze the last 5 layers
n = 5
for layer in base_model.layers[-n:]:
    layer.trainable = True

# Create the new model on top
x = base_model.output
x = Flatten()(x)
x = Dense(128, kernel_regularizer=regularizers.l2(0.01))(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, kernel_regularizer=regularizers.l2(0.01))(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x) 

predictions = Dense(3, activation='softmax')(x)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

# Set up early stopping and TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
tensorboard = TensorBoard(log_dir='logs/')

model.summary()

In [ ]:
# Train the model
model.fit(train_generator, epochs=100, validation_data=validation_generator, 
          batch_size=32, shuffle=True, callbacks=[early_stopping, tensorboard])

In [ ]:
model.save('/kaggle/working/threeemotions.h5')